In [21]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [22]:
df = pd.read_csv("sc2-matches-history.csv").dropna()

In [23]:
df.head()

,match_date,player_1,player_1_match_status,score,player_2,player_2_match_status,player_1_race,player_2_race,addon,tournament_type
0,09/19/2016,MC,[loser],0–2,Stats,[winner],P,P,LotV,online
1,09/19/2016,MC,[winner],2–1,NaTuRal,[loser],P,T,LotV,online
2,09/19/2016,MC,[loser],1–2,Dark,[winner],P,Z,LotV,online
3,09/13/2016,MC,[loser],0–2,INnoVation,[winner],P,T,LotV,online
4,08/27/2016,MC,[loser],0–1,TRUE,[winner],P,Z,LotV,online


In [24]:
df['player_1_match_status'] = df['player_1_match_status'].apply(lambda s: s == '[winner]').astype('bool')
df = df.drop('player_2_match_status', axis=1)

In [25]:
df.head()

,match_date,player_1,player_1_match_status,score,player_2,player_1_race,player_2_race,addon,tournament_type
0,09/19/2016,MC,False,0–2,Stats,P,P,LotV,online
1,09/19/2016,MC,True,2–1,NaTuRal,P,T,LotV,online
2,09/19/2016,MC,False,1–2,Dark,P,Z,LotV,online
3,09/13/2016,MC,False,0–2,INnoVation,P,T,LotV,online
4,08/27/2016,MC,False,0–1,TRUE,P,Z,LotV,online


In [26]:
df = df.drop('match_date', axis=1)

In [27]:
counts = (df['player_1'].append(df['player_2'])).value_counts()
counts_index = counts.index.tolist()
def get_player_plays(p):
    return counts[counts_index.index(p)]

In [28]:
#Leave only players who played more than 10 matches
df_players_10 = df[df['player_1'].apply(lambda p: get_player_plays(p) >= 10)][df['player_2'].apply(lambda p: get_player_plays(p) >= 10)]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [29]:
X = df_players_10.drop(['player_1_match_status', 'score'], axis=1)
y = df_players_10['player_1_match_status']

In [30]:
X.head()

,player_1,player_2,player_1_race,player_2_race,addon,tournament_type
0,MC,Stats,P,P,LotV,online
1,MC,NaTuRal,P,T,LotV,online
2,MC,Dark,P,Z,LotV,online
3,MC,INnoVation,P,T,LotV,online
4,MC,TRUE,P,Z,LotV,online


In [31]:
player_cols = ['player_1', 'player_2']
cat_cols = list(set(X.columns.values.tolist()) - set(player_cols))

In [32]:
from sklearn.feature_extraction import DictVectorizer as DV
dv = DV(sparse=False)
X_cat_oh = dv.fit_transform(X[cat_cols].T.to_dict().values())

In [33]:
X_players = X[player_cols]

In [34]:
from sklearn.model_selection import train_test_split
X_cat_tr, X_cat_ts, X_pr_tr, X_pr_ts, y_tr, y_ts = train_test_split(X_cat_oh, X_players, y, test_size=0.3)

In [47]:
param_grid = {'C': np.logspace(-2, -6, 200)}

In [43]:
X_tr = np.concatenate((X_cat_tr, X_pr_tr), axis=1)
X_ts = np.concatenate((X_cat_ts, X_pr_ts), axis=1)

In [46]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [48]:
cls = LinearSVC()
opt = GridSearchCV(cls, param_grid, cv = 3)

In [52]:
opt.fit(X_cat_tr, y_tr)

ValueError: could not convert string to float: 'SortOf'

In [51]:
opt.best_estimator_.score(X_cat_ts, y_ts)

0.53495080858101318